In [18]:
grammar = '''
{
 "grammar": {
 "-name": "G0",
 "terminalsymbols": {
 "term": [
 {
 "-name": "IDENT",
 "-spell": "a"
 },
 {
 "-name": "ADD",
 "-spell": "+"
 },
 {
 "-name": "MUL",
 "-spell": "*"
 },
 {
 "-name": "LPAREN",
 "-spell": "("
 },
 {
 "-name": "RPAREN",
 "-spell": ")"
 }
 ]
 },
 "nonterminalsymbols": {
 "nonterm": [
 { "-name": "E" },
 { "-name": "T" },
 { "-name": "F" }
 ]
 },
 "productions": {
 "production": [
 {
 "lhs": { "-name": "E" },
 "rhs": {
 "symbol": [
 {
 "-type": "nonterm",
 "-name": "E"
 },
 {
 "-type": "term",
 "-name": "ADD"
 },
 {
 "-type": "nonterm",
 "-name": "T"
 }
 ]
 }
 },
 {
 "lhs": { "-name": "E" },
 "rhs": {
 "symbol": [{
 "-type": "nonterm",
 "-name": "T"
 }]
 }
 },
 {
 "lhs": { "-name": "T" },
 "rhs": {
 "symbol": [
 {
 "-type": "nonterm",
 "-name": "T"
 },
 {
 "-type": "term",
 "-name": "MUL"
 },
 {
 "-type": "nonterm",
 "-name": "F"
 }
 ]
 }
 },
 {
 "lhs": { "-name": "T" },
 "rhs": {
 "symbol": [{
 "-type": "nonterm",
 "-name": "F"
 }]
 }
 },
 {
 "lhs": { "-name": "F" },
 "rhs": {
 "symbol": [{
 "-type": "nonterm",
 "-name": "IDENT"
 }]
 }
 },
 {
 "lhs": { "-name": "F" },
 "rhs": {
 "symbol": [
 {
 "-type": "term",
 "-name": "LPAREN"
 },
 {
 "-type": "nonterm",
 "-name": "E"
 },
 {
 "-type": "term",
 "-name": "RPAREN"
 }
 ]
 }
 }
 ]
 },
 "startsymbol": { "-name": "E" }
 }
}
'''

In [19]:
grammar = '''
{
 "grammar": {
 "-name": "G0",
 "terminalsymbols": {
 "term": [
 {
 "-name": "IDENT",
 "-spell": "a"
 },
 {
 "-name": "ADD",
 "-spell": "+"
 },
 {
 "-name": "MUL",
 "-spell": "*"
 },
 {
 "-name": "LPAREN",
 "-spell": "("
 },
 {
 "-name": "RPAREN",
 "-spell": ")"
 }
 ]
 },
 "nonterminalsymbols": {
 "nonterm": [
 { "-name": "E" },
 { "-name": "T" },
 { "-name": "F" },
 { "-name": "X" },
 { "-name": "Y" }
 ]
 },
 "productions": {
 "production": [
 {
 "lhs": { "-name": "X" },
 "rhs": {
 "symbol": [
 {
 "-type": "nonterm",
 "-name": "Y"
 }
 ]
 }
 },
 {
 "lhs": { "-name": "E" },
 "rhs": {
 "symbol": [
 {
 "-type": "nonterm",
 "-name": "E"
 },
 {
 "-type": "term",
 "-name": "ADD"
 },
 {
 "-type": "nonterm",
 "-name": "T"
 }
 ]
 }
 },
 {
 "lhs": { "-name": "E" },
 "rhs": {
 "symbol": [{
 "-type": "nonterm",
 "-name": "T"
 }]
 }
 },
 {
 "lhs": { "-name": "T" },
 "rhs": {
 "symbol": [
 {
 "-type": "nonterm",
 "-name": "T"
 },
 {
 "-type": "term",
 "-name": "MUL"
 },
 {
 "-type": "nonterm",
 "-name": "F"
 }
 ]
 }
 },
 {
 "lhs": { "-name": "T" },
 "rhs": {
 "symbol": [{
 "-type": "nonterm",
 "-name": "F"
 }]
 }
 },
 {
 "lhs": { "-name": "F" },
 "rhs": {
 "symbol": [{
 "-type": "nonterm",
 "-name": "IDENT"
 }]
 }
 },
 {
 "lhs": { "-name": "F" },
 "rhs": {
 "symbol": [
 {
 "-type": "term",
 "-name": "LPAREN"
 },
 {
 "-type": "nonterm",
 "-name": "E"
 },
 {
 "-type": "term",
 "-name": "RPAREN"
 }
 ]
 }
 }
 ]
 },
 "startsymbol": { "-name": "E" }
 }
}
'''

In [20]:
import json
grammar_dict = json.loads(grammar)

In [21]:
nonterminals = [nonterm_item['-name'] for nonterm_item in grammar_dict['grammar']['nonterminalsymbols']['nonterm']]
terminals = [term_item['-spell'] for term_item in grammar_dict['grammar']['terminalsymbols']['term']]
productions = grammar_dict['grammar']['productions']['production']

## Устранение левой рекурсии

In [22]:
new_grammar = grammar_dict.copy()
for i in range(len(nonterminals.copy())):
    #print(nonterminals[i])
    for j in range(i):
        for production in productions.copy():
            if production['lhs']['-name'] == nonterminals[i] and production['rhs']['symbol'][0]['-name'] == nonterminals[j]:
                new_grammar['grammar']['productions']['production'].remove(production)
                #print('production to delete', production)
                for other_production in productions.copy():
                    if other_production['lhs']['-name'] == nonterminals[j]:
                        new_rule = production.copy()
                        new_rule['rhs']['symbol'] = new_rule['rhs']['symbol'][1:]
                        new_rule['rhs']['symbol'] = other_production['rhs']['symbol'] + new_rule['rhs']['symbol']
                        new_grammar['grammar']['productions']['production'].append(new_rule)
                        #print('production to add ', new_rule)
    
    recursive = False
    for production in productions.copy():
        if production['lhs']['-name'] == nonterminals[i]:
            if production['lhs']['-name'] == production['rhs']['symbol'][0]['-name']:
                recursive = True
    #print('is recursive', recursive)
    if recursive:
        new_nonterm = dict()
        new_nonterm['-name'] = nonterminals[i] + 'dash'
        new_grammar['grammar']['nonterminalsymbols']['nonterm'].append(new_nonterm)
        nonterminals.append(new_nonterm['-name'])
        #print('new nonterm', new_nonterm)

        eps_rule = {'lhs': {'-name': new_nonterm}, 'rhs': {'symbol': []}}
        eps_rule['rhs']['symbol'] = list()
        #print('eps production to add ', eps_rule)
        new_grammar['grammar']['productions']['production'].append(eps_rule)
    
        for production in productions.copy():
            #print('curr production', production)
            #print('lhs', production['lhs']['-name'])
            if production['lhs']['-name'] == nonterminals[i]:
                #print('production to transform', production)
                #print('all productions',productions)
                if production['lhs']['-name'] == production['rhs']['symbol'][0]['-name']:       
                    #print('production to delete', production)
                    
                    new_grammar['grammar']['productions']['production'].remove(production)
                    
                    new_rule = production.copy()
                    new_rule['lhs']['-name'] = new_nonterm['-name']
                    new_rule['rhs']['symbol'] = new_rule['rhs']['symbol'][1:]
                    new_rule['rhs']['symbol'].append({'-type' : 'nonterm', '-name' : new_nonterm['-name']})
                    new_grammar['grammar']['productions']['production'].append(new_rule)
                    #print('step 3')
                    #print('production to add ', new_rule)
                    
                else:
                    new_rule = production.copy()
                    new_rule['rhs']['symbol'].append({'-type' : 'nonterm', '-name' : new_nonterm['-name']})
                    new_grammar['grammar']['productions']['production'].append(new_rule)
                    #print('step 2')
                    #print('production to add ', new_rule)


In [23]:
new_grammar

{'grammar': {'-name': 'G0',
  'terminalsymbols': {'term': [{'-name': 'IDENT', '-spell': 'a'},
    {'-name': 'ADD', '-spell': '+'},
    {'-name': 'MUL', '-spell': '*'},
    {'-name': 'LPAREN', '-spell': '('},
    {'-name': 'RPAREN', '-spell': ')'}]},
  'nonterminalsymbols': {'nonterm': [{'-name': 'E'},
    {'-name': 'T'},
    {'-name': 'F'},
    {'-name': 'X'},
    {'-name': 'Y'},
    {'-name': 'Edash'},
    {'-name': 'Tdash'}]},
  'productions': {'production': [{'lhs': {'-name': 'X'},
     'rhs': {'symbol': [{'-type': 'nonterm', '-name': 'Y'}]}},
    {'lhs': {'-name': 'E'},
     'rhs': {'symbol': [{'-type': 'nonterm', '-name': 'T'},
       {'-type': 'nonterm', '-name': 'Edash'}]}},
    {'lhs': {'-name': 'T'},
     'rhs': {'symbol': [{'-type': 'nonterm', '-name': 'F'},
       {'-type': 'nonterm', '-name': 'Tdash'}]}},
    {'lhs': {'-name': 'F'},
     'rhs': {'symbol': [{'-type': 'nonterm', '-name': 'IDENT'}]}},
    {'lhs': {'-name': 'F'},
     'rhs': {'symbol': [{'-type': 'term', '-name

### Устранение недостижимых символов

In [24]:
no_unreachable_grammar = new_grammar.copy()
nonterminals = [nonterm_item['-name'] for nonterm_item in no_unreachable_grammar['grammar']['nonterminalsymbols']['nonterm']]
terminals = [term_item['-spell'] for term_item in no_unreachable_grammar['grammar']['terminalsymbols']['term']]
productions = no_unreachable_grammar['grammar']['productions']['production']

In [25]:
v = list()
v.append([no_unreachable_grammar['grammar']['startsymbol']])
v[0][0]['-type'] = 'nonterm'
vi = v[0].copy()
for production in productions:
    if production['lhs']['-name'] in [lhs_sym['-name'] for lhs_sym in v[0]]:
        for symbol in production['rhs']['symbol']:
            if symbol not in vi:
                vi.append(symbol)
v.append(vi)
i = 1
while set([tuple(sym.items()) for sym in v[i]]) != set([tuple(sym.items()) for sym in v[i-1]]):
    i += 1
    vi = v[i-1].copy()
    for production in productions:
        if production['lhs']['-name'] in [lhs_sym['-name'] for lhs_sym in v[i-1]]:
            for symbol in production['rhs']['symbol']:
                if symbol not in vi:
                    vi.append(symbol)
    v.append(vi)

In [26]:
reachable_nonterms = [sym['-name'] for sym in vi if sym['-type'] == 'nonterm']
reachable_terms = [sym['-name'] for sym in vi if sym['-type'] == 'term']
for old_nonterm in no_unreachable_grammar['grammar']['nonterminalsymbols']['nonterm'].copy():
    if old_nonterm['-name'] not in reachable_nonterms:
        no_unreachable_grammar['grammar']['nonterminalsymbols']['nonterm'].remove(old_nonterm)

for old_term in no_unreachable_grammar['grammar']['terminalsymbols']['term'].copy():
    if old_term['-name'] not in reachable_terms:
        no_unreachable_grammar['grammar']['terminalsymbols']['term'].remove(old_term)

In [27]:
for production in no_unreachable_grammar['grammar']['productions']['production'].copy():
    if production['lhs']['-name'] not in reachable_nonterms:
        no_unreachable_grammar['grammar']['productions']['production'].remove(production)
        continue
    for rhs_sym in production['rhs']['symbol']:
        if rhs_sym['-type'] == 'nonterm' and rhs_sym['-name'] not in reachable_nonterms:
            no_unreachable_grammar['grammar']['productions']['production'].remove(production)
            continue
        if rhs_sym['-type'] == 'term' and rhs_sym['-name'] not in reachable_terms:
            no_unreachable_grammar['grammar']['productions']['production'].remove(production)
            continue

In [11]:
no_unreachable_grammar

{'grammar': {'-name': 'G0',
  'terminalsymbols': {'term': [{'-name': 'ADD', '-spell': '+'},
    {'-name': 'MUL', '-spell': '*'},
    {'-name': 'LPAREN', '-spell': '('},
    {'-name': 'RPAREN', '-spell': ')'}]},
  'nonterminalsymbols': {'nonterm': [{'-name': 'E'},
    {'-name': 'T'},
    {'-name': 'F'},
    {'-name': 'Edash'},
    {'-name': 'Tdash'}]},
  'productions': {'production': [{'lhs': {'-name': 'E'},
     'rhs': {'symbol': [{'-type': 'nonterm', '-name': 'T'},
       {'-type': 'nonterm', '-name': 'Edash'}]}},
    {'lhs': {'-name': 'T'},
     'rhs': {'symbol': [{'-type': 'nonterm', '-name': 'F'},
       {'-type': 'nonterm', '-name': 'Tdash'}]}},
    {'lhs': {'-name': 'F'},
     'rhs': {'symbol': [{'-type': 'nonterm', '-name': 'IDENT'}]}},
    {'lhs': {'-name': 'F'},
     'rhs': {'symbol': [{'-type': 'term', '-name': 'LPAREN'},
       {'-type': 'nonterm', '-name': 'E'},
       {'-type': 'term', '-name': 'RPAREN'}]}},
    {'lhs': {'-name': 'Edash'},
     'rhs': {'symbol': [{'-type':